In [2]:
import requests
import pandas as pd
r = requests.get("https://api.ycombinator.com/companies/export.json")
df = pd.DataFrame(r.json())

df.head()

,name,url,batch,vertical,description,dead,has_ff,all_ff
0,Thunderpod,https://www.thunderpod.com/,w2020,None,Social Fitness App where content is consumed i...,False,False,False
1,Dinesafe,https://dinesafe.org,s2018,B2B,We crowdsource food poisoning reports and help...,False,False,False
2,Parakey,http://parakey.com,s2005,None,,False,None,None
3,Demeanor.co,https://www.demeanor.co,s2018,Media,Now part of thentwrk.com,False,False,False
4,Oolu,http://oolusolar.com,s2015,Hardware,Oolu is a social enterprise that finances high...,False,True,False


In [5]:
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
import en_core_web_sm
nlp = en_core_web_sm.load()

In [101]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
#Word cloud
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

stop_words = set(stopwords.words("english"))
result = []
batches = sorted(df.batch.unique(), key=lambda x: int(x[1:])+0.1 if x[0]=='s' else int(x[1:]))
for batch in batches:
    dataset = df[df.batch == batch]
    dataset = dataset[dataset['description'].astype(bool)].reset_index()   
    corpus = []
    for i in range(0, len(dataset)-1):
        result.append((batch, nlp(dataset['description'][i])))
    words = {}
    for x in result:
        for word in list(x[1].noun_chunks):
            word = str(word)
            lem = WordNetLemmatizer()
            if lem.lemmatize(word.lower()) in stop_words:
                continue

            if word in words:
                words[word] += 1
            else:
                words[word] = 1
    wordcloud = WordCloud(
                          background_color='white',
                          max_words=100,
                          max_font_size=70, 
                          random_state=42
                         ).fit_words(words)
    wordcloud.to_file(f"result/{batch}.png")